# COMP2200 Data science group project - Group 50

#### Analysis of WHO Global Health Observatory: Communicable Diseases – Malaria

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
%matplotlib inline

In [4]:
malaria = pd.read_csv("files/malaria_data")
malaria.head()

,Country,Year,No. of cases,No. of deaths,No. of cases_median,No. of cases_min,No. of cases_max,No. of deaths_median,No. of deaths_min,No. of deaths_max,WHO Region
0,Afghanistan,2017,630308[495000-801000],298[110-510],630308,495000.0,801000.0,298,110.0,510.0,Eastern Mediterranean
1,Algeria,2017,0,0,0,NaN,NaN,0,NaN,NaN,Africa
2,Angola,2017,4615605[3106000-6661000],13316[9970-16600],4615605,3106000.0,6661000.0,13316,9970.0,16600.0,Africa
3,Argentina,2017,0,0,0,NaN,NaN,0,NaN,NaN,Americas
4,Armenia,2017,0,0,0,NaN,NaN,0,NaN,NaN,Europe


### Data cleaning

# RFE Analysis

Region vs number of infections

# K Nearest neighbour Analysis

Which variables are the most significant in relation to the spread of malaria?

# Heatmaps

Display infections in the region

# Algorithms

Model the effectiveness of preventative measures on the prevalence of the disease